In [1]:
import nyroglancer
import h5py
import os
import numpy as np
from scipy.stats import logistic
from scipy.special import logit, expit


#affs_file = '/groups/cardona/home/ecksteinn/BUNet/snapshots/run_14/batch_2501.0.hdf'
#affs_file = './predictions/run_1/70000/p_0/sample_C_padded_20160501.aligned.filled.cropped.62:153.0:268.0:268.hdf'
affs_file_0 = './predictions/run_11/96000_large/p_0/sample_C_padded_20160501.aligned.filled.cropped.62:153.hdf'
affs_file_1 = './predictions/run_11/96000_large/p_1/sample_C_padded_20160501.aligned.filled.cropped.62:153.hdf'
affs_file_2 = './predictions/run_11/96000_large/p_2/sample_C_padded_20160501.aligned.filled.cropped.62:153.hdf'

f0 = h5py.File(affs_file_0, 'r')
f1 = h5py.File(affs_file_1, 'r')
f2 = h5py.File(affs_file_2, 'r')

affs_0 = f0['volumes/labels/pred_affinities']
affs_1 = f1['volumes/labels/pred_affinities']
affs_2 = f2['volumes/labels/pred_affinities']
#sigma = np.reshape(f["volumes/labels/sigma"].value, np.shape(f["volumes/labels/pred_affinities"]))
raw = f0['volumes/raw']
#gt_affs = f["volumes/labels/affinities"]

raw = np.array(raw, dtype=np.float32)*0.5 + 0.5

voxel_size=(4,4,40)
#voxel_size=(1,1,1)
aff_offset = tuple([off for off in f0["volumes/labels/pred_affinities"].attrs.items()[0][1][::-1]])
raw_offset = tuple([off for off in f0["volumes/raw"].attrs.items()[0][1][::-1]])

#aff_offset = tuple([0,0,0])
#raw_offset = tuple([0,0,0])

shader = """void main() { 
  vec3 color = vec3(toNormalized(getDataValue(0)*10.),
                    toNormalized(getDataValue(1)*10.),
                    toNormalized(10.*getDataValue(2)));
  if (color[0] > 0.3
      color[1] > 0.3 
      && color[2] > 0.3) 
  emitRGB(vec3(color[0],color[1],color[2]));}"""

ale_shader = """void main() { 
  vec3 color = vec3(toNormalized(getDataValue(0)*100.),
                    toNormalized(getDataValue(1)*100.),
                    toNormalized(getDataValue(2)*100.));
  emitRGB(vec3(color[0],color[1],color[2]));}"""

#ale = abs(expit(logit(affs) + sigma) - expit(logit(affs) - sigma)) 

viewer = nyroglancer.Viewer()
viewer.add(raw, name='raw', voxel_size=voxel_size, offset=raw_offset)
viewer.add(affs_0, name='predicted0', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
viewer.add(affs_1, name='predicted1', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
viewer.add(affs_2, name='predicted2', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
#viewer.add(ale, name='aleatoric_prop', voxel_size=voxel_size, shader=ale_shader, offset=aff_offset)
#viewer.add(sigma, name='aleatoric', voxel_size=voxel_size, shader=ale_shader, offset=aff_offset)
#viewer.add(gt_affs, name='gt_affs', voxel_size=voxel_size, shader=nyroglancer.shaders.rgb(), offset=aff_offset)
viewer.set_hostname('localhost:8880')
viewer.show()